# Import of the libraries

In [1]:
import optuna
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import tqdm
from sklearn.metrics import roc_curve, auc

# We run the code in GPU using cuda

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
dimensionality = [9, 12, 15, 18]

Using device: cuda


# Definition of NN model

In [3]:
class DynamicNeuralNetwork(torch.nn.Module):
    def __init__(self, dim, hidden_sizes, dropout_rate):
        super(DynamicNeuralNetwork, self).__init__()
        layers = []
        last_size = dim

        for size in hidden_sizes:
            layers.append(torch.nn.Linear(last_size, size))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(dropout_rate))
            last_size = size

        layers.append(torch.nn.Linear(last_size, 1))
        layers.append(torch.nn.Sigmoid())
        self.network = torch.nn.Sequential(*layers)

        # Initialize weights, based on https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/Basics/pytorch_init_weights.py
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                torch.nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        return self.network(x)

# Pre-processing and data loaders defintion

In [4]:
def denoise_bernoulli_data(X):
    """Denoise data by converting to binary values"""
    return (X >= 0.5).astype(float)

In [5]:
def create_dataloaders(X, y, batch_size):
    """Convert numpy arrays to PyTorch tensors and create DataLoaders."""
    X = denoise_bernoulli_data(X)
    X = torch.tensor(X, dtype=torch.float32).to(device)
    y = torch.tensor(y, dtype=torch.long).to(device)

    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    return loader

# Function that spcifies how the model is trained

In [6]:
def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device, early_stopping_patience=50, print_statements=False):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    patience_counter = 0
    
    best_val_loss = float('inf')
    for epoch in tqdm.tqdm(range(num_epochs)):
        # Train Model
        model.to(device)
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Calculate accuracy
            predicted = (outputs >= 0.5).float()
            train_correct += (predicted == labels.unsqueeze(1)).sum().item()
            train_total += labels.size(0)
        
        train_loss = train_loss / len(train_loader)
        train_accuracy = train_correct / train_total * 100
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Evaluate the model
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels.unsqueeze(1).float())
                
                val_loss += loss.item()

                # Calculate accuracy
                predicted = (outputs >= 0.5).float()
                val_correct += (predicted == labels.unsqueeze(1)).sum().item()
                val_total += labels.size(0)

        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1


        if patience_counter >= early_stopping_patience and print_statements:
            print(f'Early stopping at epoch {epoch + 1}')
            break
        
        if (epoch + 1) % 10 == 0 and print_statements:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    return model, best_val_loss, train_losses, val_losses, train_accuracies, val_accuracies

# Evaluation of the trained model using test loader definition

In [7]:
def evaluate_model(model, dim, test_loader, save_file):
	model.eval()
	all_preds = []
	all_labels = []
    
	with torch.no_grad():
		for inputs, labels in test_loader:
			inputs = inputs.view(inputs.size(0), -1)
			outputs = model(inputs)
			predictions = (outputs >= 0.5).float().cpu().numpy()
			all_preds.extend(predictions)
			all_labels.extend(labels.cpu().numpy())
    
	all_preds = np.array(all_preds).squeeze()
	print(all_preds)
	
	if save_file:    
		all_labels = np.array(all_labels)
		np.save(f'kryptonite_{dim}_pred_nn.npy', all_preds)

	return {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds),
        'recall': recall_score(all_labels, all_preds),
        'f1': f1_score(all_labels, all_preds)
    }

# Functions to get predictions

- get_predictions is mostly used for the model evaluation.
- get_predictions_final is used for the predictions on the hidden data.
- get_predictions_prob is used to get the probability for the sigmoid link function. This is used for the AUC graph and ROC scores.

In [8]:
def get_predictions(model, test_loader):
	model.eval()
	all_preds = []
	all_labels = []
    
	with torch.no_grad():
		for inputs, labels in test_loader:
			inputs = inputs.view(inputs.size(0), -1)
			outputs = model(inputs).float().cpu().numpy()
			predictions = (outputs >= 0.5).float().cpu().numpy()
			all_preds.extend(predictions)
			all_labels.extend(labels.cpu().numpy())

	all_preds = np.array(all_preds).squeeze()
	print(all_preds)

	return {
		'all_predictions': all_preds,
		'all_labels': all_labels
    }

In [9]:
def get_predictions_final(model, test_loader):
	model.eval()
	all_preds = []

	with torch.no_grad():
		for inputs in test_loader:
			inputs = inputs.view(inputs.size(0), -1)
			outputs = model(inputs).float().cpu().numpy()
			predictions = (outputs >= 0.5).float().cpu().numpy()
			all_preds.extend(predictions)

	all_preds = np.array(all_preds).squeeze()
	print(all_preds)

	return {
		'all_predictions': all_preds
    }

In [10]:
def get_predictions_prob(model, test_loader):
	model.eval()
	all_preds = []
	all_labels = []

	with torch.no_grad():
		for inputs, labels in test_loader:
			inputs = inputs.view(inputs.size(0), -1)
			outputs = model(inputs).float().cpu().numpy()
			all_preds.extend(outputs)
			all_labels.extend(labels.cpu().numpy())
    
	all_preds = np.array(all_preds).squeeze()
	print(all_preds)

	return {
		'all_predictions': all_preds,
		'all_labels': all_labels
    }

# Defintion of cross validation function

In [11]:
def k_fold_cross_validation(X, y, best_params, dim, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    results = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train_loader = create_dataloaders(X_train, y_train, best_params['batch_size'])
        val_loader = create_dataloaders(X_test, y_test, best_params['batch_size'])

        model = DynamicNeuralNetwork(dim, best_params['hidden_sizes'], best_params['dropout_rate']).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
        loss_fn = torch.nn.BCELoss()

        trained_model, _, _, _, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, 100, device)
        results.append(evaluate_model(trained_model, dim, val_loader, False))

    metric_names = ['accuracy', 'precision', 'recall', 'f1']
    avg_metrics = {
        metric: np.mean([result[metric] for result in results])
        for metric in metric_names
    }
    
    std_metrics = {
        metric: np.std([result[metric] for result in results])
        for metric in metric_names
    }
    
    print("\nAverage Results Across All Folds:")
    for metric in metric_names:
        print(f"{metric.capitalize()}: {avg_metrics[metric]:.4f} ± {std_metrics[metric]:.4f}")
    
    return avg_metrics, std_metrics

# Defintion of the objective function we minize in the Optuna framework

This function is used to find the best hyper paramters (it performs the hyper parameter search)

In [14]:
def objective(trial, dim):
	X_train = np.load(f'../../Datasets_Train_Test_Split/kryptonite-{dim}-X_train.npy')
	y_train = np.load(f'../../Datasets_Train_Test_Split/kryptonite-{dim}-y_train.npy')

	X_train = denoise_bernoulli_data(X_train)

	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

	batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
	learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.01, step=0.001)
	weight_decay = 0.0001
	dropout_rate = 0.15
	num_layers = trial.suggest_int('num_layers', 2, 4)
	hidden_sizes = [trial.suggest_categorical(f'hidden_size_l{i}', [64, 128, 256]) for i in range(num_layers)]
	num_epochs = 200

	train_loader = create_dataloaders(X_train, y_train, batch_size)
	val_loader = create_dataloaders(X_val, y_val, batch_size)
	model = DynamicNeuralNetwork(dim, hidden_sizes=hidden_sizes, dropout_rate=dropout_rate).to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	loss_fn = torch.nn.BCELoss()
	_, _, _, val_losses, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device)

	return val_losses[-1]

In [15]:
best_params_by_dim = []

for dim in dimensionality:
	print(f"-=-=-=-= Running Dimension: {dim} =-=-=-=-")

	study = optuna.create_study(direction='minimize')
	study.optimize(lambda trial: objective(trial, dim), n_trials=10)

	print("Best trial:")
	trial = study.best_trial

	print(f"  Value: {trial.value}")
	print("  Params: ")

	for key, value in trial.params.items():
		print(f"    {key}: {value}")

	best_params = trial.params
	best_params['hidden_sizes'] = [best_params.pop(f'hidden_size_l{i}') for i in range(best_params['num_layers'])]

	del best_params['num_layers']
	best_params_by_dim.append(best_params)

[I 2024-11-25 17:49:52,325] A new study created in memory with name: no-name-e608ba07-6760-46df-b01f-fd3aedb383c6


-=-=-=-= Running Dimension: 9 =-=-=-=-


c:\Users\migue\anaconda3\envs\MML-CW\Lib\site-packages\optuna\distributions.py:693: UserWarning: The distribution is specified by [0.0001, 0.01] and step=0.001, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0091].
  warnings.warn(
  2%|▏         | 3/200 [00:06<07:31,  2.29s/it]
[W 2024-11-25 17:50:01,636] Trial 0 failed with parameters: {'batch_size': 16, 'learning_rate': 0.0001, 'num_layers': 3, 'hidden_size_l0': 64, 'hidden_size_l1': 256, 'hidden_size_l2': 128} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\migue\anaconda3\envs\MML-CW\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\migue\AppData\Local\Temp\ipykernel_20368\3240112142.py", line 7, in <lambda>
    study.optimize(lambda trial: objective(trial, dim), n_trials=10)
                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Us

KeyboardInterrupt: 

# Defintions of functions to plot the AUC and get ROC score

- plot_roc_curve plots one graph for one dimesnion
- plot_multi_roc_curves plots multiple AUCs for multiple dimensions in one graph

In [16]:
def plot_roc_curve(y_true, y_pred_proba):
    """
    Plot ROC curve and calculate AUC score from predictions and true values.
    
    Parameters:
    -----------
    y_true : array-like
        True binary labels (0, 1)
    y_pred_proba : array-like
        Predicted probabilities for the positive class
        
    Returns:
    --------
    float
        AUC score
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.fill_between(fpr, tpr, alpha=0.2, color='darkorange')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    
    return roc_auc

In [17]:
def plot_multi_roc_curves(y_true_list, y_pred_proba_list, dimensions):
    """
    Plot ROC curves for multiple dimensions on the same figure.
    
    Parameters:
    -----------
    y_true_list : list of array-like
        List of true binary labels for each dimension
    y_pred_proba_list : list of array-like
        List of predicted probabilities for each dimension
    dimensions : list
        List of dimension values
    
    Returns:
    --------
    dict
        Dictionary of AUC scores for each dimension
    """
    plt.figure(figsize=(10, 8))
    auc_scores = {}
    
    colors = plt.cm.rainbow(np.linspace(0, 1, len(dimensions)))
    
    for (y_true, y_pred_proba, dim, color) in zip(y_true_list, y_pred_proba_list, dimensions, colors):
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        auc_scores[dim] = roc_auc
        
        plt.plot(fpr, tpr, color=color, lw=2, 
                label=f'Dim {dim} (AUC = {roc_auc:.2f})')
        plt.fill_between(fpr, tpr, alpha=0.1, color=color)
    
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    plt.xlabel('False Positive Rate', fontsize=22)
    plt.ylabel('True Positive Rate', fontsize=22)
    plt.title('ROC Curves for Different Dimensions', fontsize=22)
    plt.legend(loc="lower right", bbox_to_anchor=(1.15, 0), fontsize=22)
    plt.grid(True, alpha=0.3)
    
    return auc_scores

# Script to plot AUC curves

In [20]:
best_parameters = {
    'batch_size': 16,
    'learning_rate': 0.001,
    'weight_decay': 0.0001,
    'dropout_rate': 0.15,
    'num_layers': 4,  # Assuming max suggested layers since specific layer count not given
    'hidden_sizes': [256, 256, 256, 256],  # Assuming max suggested size for each layer since specific sizes not given
    'num_epochs': 1000,
    'momentum': 0.9,
    'early_stopping_patience': 50
}

best_params_by_dim = [best_parameters,best_parameters,best_parameters,best_parameters,best_parameters]


In [23]:
y_true_list = []
y_pred_proba_list = []

for i, dim in enumerate(dimensionality):
	print(f"-=-=-=-= Running {dim} =-=-=-=-")

	X_train = np.load(f'../../Datasets_Train_Test_Split/kryptonite-{dim}-X_train.npy')
	X_test = np.load(f'../../Datasets_Train_Test_Split/kryptonite-{dim}-X_test.npy')
	y_train = np.load(f'../../Datasets_Train_Test_Split/kryptonite-{dim}-y_train.npy')
	y_test = np.load(f'../../Datasets_Train_Test_Split/kryptonite-{dim}-y_test.npy')

	X_train = denoise_bernoulli_data(X_train)
	X_test = denoise_bernoulli_data(X_test)

	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
	best_params = best_params_by_dim[i]
	train_loader = create_dataloaders(X_train, y_train, best_params['batch_size'])
	val_loader = create_dataloaders(X_val, y_val, best_params['batch_size'])
	test_loader = create_dataloaders(X_test, y_test, best_params['batch_size'])
	model = DynamicNeuralNetwork(dim, best_params['hidden_sizes'], 0.15).to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=0.0001)
	loss_fn = torch.nn.BCELoss()
	trained_model, _, _, _, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, 200, device)
	test_metrics_run = evaluate_model(trained_model, dim, test_loader, False)

	for metric, value in test_metrics_run.items():
		print(f"{metric.capitalize()}: {value:.4f}")

	predictions = get_predictions_prob(trained_model, test_loader)
	y_true_list.append(predictions['all_labels'])
	y_pred_proba_list.append(predictions['all_predictions'])

	plot_roc_curve(predictions['all_labels'], predictions['all_predictions'])

-=-=-=-= Running 9 =-=-=-=-


  2%|▏         | 3/200 [00:07<08:38,  2.63s/it]


KeyboardInterrupt: 

In [ ]:
plot_multi_roc_curves(y_true_list, y_pred_proba_list, dimensionality)

# Script to run cross validation

In [26]:
for i, dim in enumerate(dimensionality):
	print(f"-=-=-=-= Running {dim} =-=-=-=-")

	best_params = best_params_by_dim[i]

	X = np.load(f'../../Datasets/kryptonite-{dim}-X.npy')
	y = np.load(f'../../Datasets/kryptonite-{dim}-y.npy')

	X = denoise_bernoulli_data(X)

	# Get both metrics and standard deviations
	avg_metrics, std_metrics = k_fold_cross_validation(X, y, best_params, dim, k=2)

	print("\nFinal K-Fold Cross Validation Results:")
	print(f"Average Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
	print(f"Average Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
	print(f"Average Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
	print(f"Average F1: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")

-=-=-=-= Running 9 =-=-=-=-


 15%|█▌        | 15/100 [00:36<03:25,  2.42s/it]


KeyboardInterrupt: 

# Getting the predicitons of the hidden data up to 18 dimesnions

In [ ]:
dimensionality = [9, 12, 15, 18]

for i, dim in enumerate(dimensionality):
	print(f"-=-=-=-= Running {dim} =-=-=-=-")
	X = np.load(f'..\..\Datasets\kryptonite-{dim}-X.npy')
	y = np.load(f'..\..\Datasets\kryptonite-{dim}-y.npy')

	X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

	X_train = np.array(denoise_bernoulli_data(X_train))
	X_val = np.array(denoise_bernoulli_data(X_val))

	best_params = best_params_by_dim[i] # The best parameter for dim = 18 is used for the higher dimensions

	# Convert to PyTorch tensors
	X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
	y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
	X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
	y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

	# Create data loaders
	train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
	val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
	train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
	val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False)

	# Initialize and train model
	model = DynamicNeuralNetwork(dim, best_params['hidden_sizes'], 0.15).to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=0.0001)
	loss_fn = torch.nn.BCELoss()
	trained_model, _, _, _, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, 200, device)

	X = np.load(rf'..\Kryptonite-N-main\Datasets\hidden-kryptonite-{dim}-X.npy')
	X = denoise_bernoulli_data(X)
	X = torch.tensor(X, dtype=torch.float32).to(device)
    
	predict_dataset = TensorDataset(X)
	predict_loader = DataLoader(predict_dataset, batch_size=best_params['batch_size'], shuffle=False)
    
	predictions = get_predictions_final(model, predict_loader)
	np.save(f'nn_predictions_{dim}.npy', predictions)

<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
C:\Users\migue\AppData\Local\Temp\ipykernel_20368\1454967782.py:5: SyntaxWarning: invalid escape sequence '\.'
  X = np.load(f'..\..\Datasets\kryptonite-{dim}-X.npy')
C:\Users\migue\AppData\Local\Temp\ipykernel_20368\1454967782.py:6: SyntaxWarning: invalid escape sequence '\.'
  y = np.load(f'..\..\Datasets\kryptonite-{dim}-y.npy')


-=-=-=-= Running 9 =-=-=-=-


 32%|███▏      | 64/200 [03:04<07:09,  3.16s/it]

# Getting the predicitons of the hidden data dim > 18

In [ ]:
dimensionality = [24, 30, 45]

for i, dim in enumerate(dimensionality):
	print(f"-=-=-=-= Running {dim} =-=-=-=-")
	X = np.load(rf'..\Kryptonite-N-main\Datasets\kryptonite-{dim}-X.npy')
	y = np.load(rf'..\Kryptonite-N-main\Datasets\kryptonite-{dim}-y.npy')

	X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

	X_train = np.array(denoise_bernoulli_data(X_train))
	X_val = np.array(denoise_bernoulli_data(X_val))

	best_params = best_params_by_dim[i]

	# Convert to PyTorch tensors
	X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
	y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
	X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
	y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

	# Create data loaders
	train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
	val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
	train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
	val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False)

	# Initialize and train model
	model = DynamicNeuralNetwork(dim, best_params['hidden_sizes'], 0.15).to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=0.0001)
	loss_fn = torch.nn.BCELoss()
	trained_model, _, _, _, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, 200, device)

	X = np.load(rf'..\Kryptonite-N-main\Datasets\hidden-kryptonite-{dim}-X.npy')
	X = denoise_bernoulli_data(X)
	X = torch.tensor(X, dtype=torch.float32).to(device)
    
	predict_dataset = TensorDataset(X)
	predict_loader = DataLoader(predict_dataset, batch_size=best_params['batch_size'], shuffle=False)
    
	predictions = get_predictions_final(model, predict_loader)
	np.save(f'nn_predictions_{dim}.npy', predictions)